In [ ]:
import random

In [ ]:
GAS_LEVEL = 100
POWER_LEVEL = 100
FOOD_LEVEL = 100
WATER_LEVEL = 100
CREW_NUMBER = 6
FOOD_NEED = 20
WATER_NEED = 20
PROB_STATION = 0.045
PROB_ATTACK = 0.05
PROB_BEACON = 0.035
PROB_TRADE = 0.06
PROB_EXPLORATION = 0.05
PROB_SOS = 0.035

In [ ]:
class Game(object):
    def __init__(self):
        self.options = ["Gas", "Power", "Food", "Water"]
        self.control_types = ["Standard", "Gas", "Electrical"]
        self.resources = [GAS_LEVEL, POWER_LEVEL, FOOD_LEVEL, WATER_LEVEL] # 0-gas, 1-power, 2-food, 3-water
        self.crew = CREW_NUMBER
        self.chance_station = PROB_STATION
        self.chance_attack = PROB_ATTACK
        self.chance_beacon = PROB_BEACON
        self.chance_trade = PROB_TRADE
        self.chance_exploration = PROB_EXPLORATION
        self.chance_sos = PROB_SOS
        self.control_mode = 0
        self.points = 0
        self.events_prob = [0, 0, 0, 0, 0, 0] # [station, attack, beacon, trade, exploration, sos]
        self.options_station_event = [0,0] # 0 = power, 1 = gas, 2 = food, 3 = water
        self.resources_trade_event = [0,0] # [resource requested, resource offered]
        self.crew_exploration_event = 0
        self.crew_sos_event = 0
        self.gameover = False
        self.selected_event = None
        self.select_events()
        
    #This function will select an event every turn
    def select_events(self):
        self.selected_event = None
        
        station = random.random()
        attack = random.random()
        beacon = random.random()
        trade = random.random()
        exploration = random.random()
        sos = random.random()
        
        events_selected = 0
        
        if station < self.chance_station:
            self.events_prob[0] = 1
            events_selected += 1
        else:
            self.events_prob[0] = 0
            
        if attack < self.chance_attack:
            self.events_prob[1] = 1
            events_selected += 1
        else:
            self.events_prob[1] = 0
            
        if beacon < self.chance_beacon:
            self.events_prob[2] = 1
            events_selected += 1
        else:
            self.events_prob[2] = 0
            
        if trade < self.chance_trade:
            self.events_prob[3] = 1
            events_selected += 1
        else:
            self.events_prob[3] = 0
            
        if exploration < self.chance_exploration:
            self.events_prob[4] = 1
            events_selected += 1
        else:
            self.events_prob[4] = 0
            
        if sos < self.chance_sos:
            self.events_prob[5] = 1
            events_selected += 1
        else:
            self.events_prob[5] = 0
            
        if events_selected != 0 and events_selected != 1:
            events_to_randomize = []
            for i in range(len(self.events_prob)):
                if self.events_prob[i] == 1:
                    events_to_randomize.append(i)
                    
            randomize = random.randint(0, len(events_to_randomize)-1)
            
            for i in range(len(self.events_prob)):
                if i != events_to_randomize[randomize]:
                    self.events_prob[i] = 0
                    
            events_to_randomize.clear()
            
        for i in range(len(self.events_prob)):
            if self.events_prob[i] == 1:
                self.selected_event = i
                
        if self.selected_event == None:
            self.select_events()
        
        self.options_station_event = [0,0]
        self.resources_trade_event = [0,0] 
        self.crew_exploration_event = 0
        self.crew_sos_event = 0
            
        self.select_event_random_values(self.selected_event)
        
    #This function will select random values for the random event selected, e.g: The resources offered in the 
    #"Station event", the number of crewmates for the "Exploration event"
    def select_event_random_values(self, event):
        #station event
        if event == 0:
            self.options_station_event[0] = random.randint(0, 3)
            self.options_station_event[1] = random.randint(0, 3)
            
            while self.options_station_event[1] == self.options_station_event[0]:
                self.options_station_event[1] = random.randint(0, 3)
        
        #trade event
        elif event == 3:
            self.resources_trade_event[0] = random.randint(0, 3)
            self.resources_trade_event[1] = random.randint(0, 3)
            
            while self.resources_trade_event[1] == self.resources_trade_event[0]:
                self.resources_trade_event[1] = random.randint(0, 3)
        
        #exploration event
        elif event == 4:
            self.crew_exploration_event = random.randint(1, 4)
            while self.crew - self.crew_exploration_event < 0:
                self.crew_exploration_event = 0
                self.crew_exploration_event = random.randint(1, 4)
            
        #sos event
        elif event == 5:
            self.crew_sos_event = random.randint(1, 5)
            
    #Check which event has been selected and call the respective event's function
    def check_events(self, crewmates):
        if self.events_prob[0] == 1:
            self.station_event()
            
        if self.events_prob[1] == 1:
            self.attack_event()
            
        if self.events_prob[2] == 1:
            self.beacon_event()
            
        if self.events_prob[3] == 1:
            self.trade_event()
            
        if self.events_prob[4] == 1:
            self.exploration_event(crewmates)
            
        if self.events_prob[5] == 1:
            self.sos_event(crewmates)
    
    # Controls the power and gas depletion
    def ship(self):
        if self.control_mode == 0:
            self.resources[1] -= 2
            self.resources[0] -= 2
            return
            
        elif self.control_mode == 1:
            self.resources[0] -= 4
            return
            
        elif self.control_mode == 2:
            self.resources[1] -= 4
            return
            
    def change_mode(self):
        decision = 0
        
        while int(decision) == 0:
            decision = input("\nShould the space change"
                             +" the propulsion system? Current: {}".format(self.control_types[self.control_mode])
                             +"\n[1] Standard [2] Gas [3] Electrical: ")
            
            if str(decision).isdigit():
                if int(decision) == 1 or int(decision) == 2 or int(decision) == 3:
                    decision = decision

                else:
                    decision = 0

            else:
                decision = 0

        if int(decision) == 1:
            self.control_mode = 0
            return

        elif int(decision) == 2:
            self.control_mode = 1
            return

        else:
            self.control_mode = 2
            return
                
    # Code for supply replenishment event
    def station_event(self):
        decision = 0
        
        while int(decision) == 0:
            decision = input("\nYour crew passes near a Space Station. The Station offers to refill some of your supplies."
                             + "\n[1] {} [2] {}: ".format(self.options[self.options_station_event[0]], 
                                                          self.options[self.options_station_event[1]]))
            
            if str(decision).isdigit():
                if int(decision) == 1 or int(decision) == 2:
                    decision = decision
                    
                else:
                    decision = 0

            else:
                decision = 0
        
        if int(decision) == 1:
            decision = self.options_station_event[0]
            
        elif int(decision) == 2:
            decision = self.options_station_event[1]
            
        refill = random.randint(20, 50)
            
        self.update_post_event(decision, refill)
            
        input("The Space Station crew refilled {} units of {}!".format(refill, self.options[decision].lower())
             +" (Press enter to continue)")
        return
        
    # Code for event space pirate attack
    def attack_event(self):
        decision = input("\nOh no! There is a space pirate fleet nearby! Should the crew boost the engines to scape?"
                         +"\n[Y] (Use more fuel to escape) / [N] (Small chance to escape unharmed): ")
        if decision.upper() == "YES" or decision.upper() == "Y":
            depletion = random.randint(15, 35)
            self.resources[0] -= depletion
            input("Using {} extra fuel, your crew escaped successfully!".format(depletion)
                 +" (Press enter to continue)")
            return
                
        if decision.upper() == "NO" or decision.upper() == "N":
            escape_prob = 0.4
            escape_try = random.random()
            
            if escape_prob >= escape_try:
                input("Fortunately the pirates miss your spaceship." 
                      + " The crew waits until they are gone and resume their trip."
                      +" (Press enter to continue)")
                return
                
            else:
                supply_lost = random.randint(0, 3)
                amount_lost = -1 * random.randint(30, 50)
                
                self.update_post_event(supply_lost, amount_lost)
                
                input("Oh no! The pirates raided the spaceship and took some of your supplies." 
                      +" You lost {} {}".format(-1 * amount_lost, self.options[supply_lost])
                      +" (Press enter to continue)")
            
                return
        
        else:
            self.attack_event()
        
    # Code for supply beacon event
    def beacon_event(self):        
        decision = input("\nAs the crew is travelling through space, they suddenly stumble upon a supply beacon!"
                        + " Do you want to open it? \n[Y]/[N]: ")
        if decision.upper() == "YES" or decision.upper() == "Y":
            supply = random.randint(0, 3)
            amount_found = random.randint(10, 50)
            
            input("You found {} {} units!".format(amount_found, self.options[supply])
                  +" (Press enter to continue)")
            
            self.update_post_event(supply, amount_found)
            
            return
                
        if decision.upper() == "NO" or decision.upper() == "N":
            input("You decided to leave the beacon as it is. Maybe someone else needs it more than you."
                  +" (Press enter to continue)")
            return
        
        else:
            self.beacon_event()

    # Code for trade event
    def trade_event(self):
        amount_offered = random.randint(30, 60)
        amount_requested = random.randint(10, 40)
        
        decision = input("\nA friendly spaceship approaches you! They offer" 
                         + " to give you {} {} for {} {}.".format(amount_offered, self.options[self.resources_trade_event[0]], 
                                                                  amount_requested, self.options[self.resources_trade_event[1]])
                         + " What do you say?\n[Y]/[N]: ")
        
        if decision.upper() == "YES" or decision.upper() == "Y":
            input("You accept the exchange and start the transfer process. The other crew seems relieved."
                  +" (Press enter to continue)")
            self.update_post_event(self.resources_trade_event[0], amount_offered)
            self.update_post_event(self.resources_trade_event[1], -1 * amount_requested)
            return
                
        if decision.upper() == "NO" or decision.upper() == "N":
            input("You decided not to trade. The other spaceship seems disappointed, but leaves."
                  +" (Press enter to continue)")
            return
        
        else:
            self.trade_event()
            
    # Code for exploration event
    def exploration_event(self, crewmates):
        sucess_chance = 0.65
        
        decision = input("\nThere is a planet within the range of your spaceship." 
                         +" {} crewmates can leave the ship to search".format(self.crew_exploration_event)
                         +" for resources. However, they might not return. Should they go? \n [Y]/[N]: ")
        
        if decision.upper() == "YES" or decision.upper() == "Y":
            print("{} brave crewmates decided to go explore the planet for resources".format(self.crew_exploration_event))
            exploration_result = random.random()
            if sucess_chance > exploration_result:
                found = random.randint(0, 3)
                amount_found = random.randint(50, 80)
                input("Several hours pass by. As the crew starts to get worried, you hear a signal."
                     + "The crewmates who had left are coming back with {} {}!".format(amount_found, 
                                                                                       self.options[found])
                      +" (Press enter to continue)")
                
                self.update_post_event(found, amount_found)
                return
            
            else:
                crewmates.lost_in_planet(self.crew_exploration_event)
                for i in range(self.crew_exploration_event):
                    self.crew -= 1
                    self.points -= 500
                    
                input("You wait as the minutes pass by. Soon the minutes turn into hours, and then days pass by without"
                     +" any communication from the team in the planet. The crew decides to vote on their next action, and"
                     +" soon after the spaceship turns on the engine again. They never came back."
                     +" (Press enter to continue)")
                return
                
        if decision.upper() == "NO" or decision.upper() == "N":
            input("You decided it isn't worth the risk. Some of you watch as the planet disappears into the distance"
                 +" (Press enter to continue)")
            return
        
        else:
            self.exploration_event(crewmates)

    # Code for sos call event
    def sos_event(self, crewmates):
        decision = input("\nYou were attending your duties when a crewmember calls you to the bridge. It seems like there"
                        + " was an accident nearby and {} persons are asking for help.".format(self.crew_sos_event)
                        + " Should you take them onboard?\n[Y]/[N]: ")
        
        if decision.upper() == "YES" or decision.upper() == "Y":
            input("You quickly respond to their call and head to their location to rescue them. Your crew gained"
                 +" {} more passengers".format(self.crew_sos_event)
                 +" (Press enter to continue)")
            
            new_crew = self.crew + self.crew_sos_event
            
            for i in range(self.crew, new_crew):
                crewmates.crewmate()
                
            self.crew = new_crew
            
            return
                
        if decision.upper() == "NO" or decision.upper() == "N":
            input("You decided that the best will be to leave them. Hopefully someone else can help them"
                 +" (Press enter to continue)")
            return
        
        else:
            self.sos_event(crewmates)
    
    # Code for meal event
    def meal(self, crewmates):
        ask = input("\nShould the crew go eat? \n[Y]/[N]: ")
        if ask.upper() == "YES" or ask.upper() == "Y":
            for i in range(len(crewmates.crewmates_left)):
                if not crewmates.crewmates_left[i]:
                    self.give_food(i, crewmates, None)
                    self.give_water(i, crewmates, None)
            return
                
        if ask.upper() == "NO" or ask.upper() == "N":
            return
        
        else:
            self.meal(crewmates)
        
    # Code for giving food to the crew
    def give_food(self, name, crewmates, food):
        if self.resources[2] > 5:
            self.resources[2] -= 6
            crewmates.crewmates_food[name] += 6
        return
            
    # Code for giving water to the crew
    def give_water(self, name, crewmates, water):
        if self.resources[3] > 5:
            self.resources[3] -= 6
            crewmates.crewmates_water[name] += 6
        return
    
    def update_post_event(self, resource, amount):
        if resource == 0:
            self.resources[0] += amount

        elif resource == 1:
            self.resources[1] += amount

        elif resource == 2:
            self.resources[2] += amount
            if self.resources[2] < 0:
                self.resources[2] = 0

        elif resource == 3:
            self.resources[3] += amount
            if self.resources[3] < 0:
                self.resources[3] = 0
    
    # Checks if a game over condition is met
    def check_game_over(self):
        if self.resources[0] <= 0:
            self.gameover = True
            print("Game Over - You ran out of gas")
            return
            
        if self.resources[1] <= 0:
            self.gameover = True
            print("Game Over - You ran out of power")
            return
        
        if self.crew == 0:
            self.gameover = True
            print("Game Over - All crewmates left the spaceship")
            return
        
        else:
            return

In [ ]:
class Crewmate(object):
    def __init__(self):
        self.crewmates_food = []
        self.crewmates_water = []
        self.crewmates_left = []
        for i in range(CREW_NUMBER):
            self.crewmate()
        
    # Generates a crewmate
    def crewmate(self):
        self.crewmates_food.append(FOOD_NEED)
        self.crewmates_water.append(WATER_NEED)
        self.crewmates_left.append(False)
    
    def crewmates_status(self, name):
        if not self.crewmates_left[name]:
            print("Crewmate {} - Food: {} Water: {}".format(name, self.crewmates_food[name], 
                                                            self.crewmates_water[name]))
            
        if self.crewmates_left[name]:
            print("Crewmate {} isn't in the spaceship anymore".format(name))
    
    # Updates each crewmate's food and water levels
    def run_crewmates(self, name, game):
        if not self.crewmates_left[name]:
            if self.crewmates_food[name] == 0:
                print("Crewmate {} didn't get enough food. Leaves the spaceship".format(name))
                game.crew -= 1
                self.crewmates_left[name] = True
                game.points -= 500
                return

            elif self.crewmates_water[name] == 0:
                print("Crewmate {} didn't get enough water. Leaves the spaceship".format(name))
                game.crew -= 1
                self.crewmates_left[name] = True
                game.points -= 500
                return
            
            else:
                self.crewmates_food[name] -= 1
                self.crewmates_water[name] -= 1

    # Code for crewmates lost in exploration event
    def lost_in_planet(self, number):
        pool = []
        
        for i in range(len(self.crewmates_left)):
            if not self.crewmates_left[i]:
                pool.append(i)
        
        len_pool = len(pool)
        for i in range(number):
            name = random.randint(0, len_pool - 1)
            while self.crewmates_left[pool[name]] == True:
                name = random.randint(0, len_pool - 1)

            self.crewmates_left[pool[name]] = True

In [ ]:
def run_game(game, crewmates):
    turn = 0
    
    while not game.gameover:
        print("Turn {}".format(turn), "Power supply:", game.resources[1], "|", "Gas supply:", game.resources[0],
             "|", "Food supply:", game.resources[2], "|", "Water supply:", game.resources[3])
        
        for i in range(len(crewmates.crewmates_left)):
            crewmates.crewmates_status(i)
            
        if turn % 5 == 0 and turn != 0:
            game.change_mode()
        
        if turn % 6 == 0 and turn != 0:
            game.meal(crewmates)
        
        game.check_events(crewmates)
        
        game.ship()
                
        for i in range(len(crewmates.crewmates_left)):
            crewmates.run_crewmates(i, game)
        
        game.check_game_over()
        
        if game.gameover == True:
            return
        
        for i in range(len(crewmates.crewmates_left)):
            if crewmates.crewmates_left[i] == False:
                game.points += 100
        
        turn += 1
        
        game.select_events()
        
        print("----------------------------------------------------------------------------------------------")

    return

In [ ]:
def main():
    game = Game()
    crewmates = Crewmate()
    
    run_game(game, crewmates)
    
    print("Final score {}".format(game.points))
    
    f= open("Highscores.txt", "a")
    
    name = input("Name: ")
    
    f.write("{}: {}\n".format(name, game.points))
    
    f.close()
    
if __name__ == '__main__':
    main()